In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
tqdm.pandas()

In [4]:
df1 = pd.read_csv(
    Path(os.getenv("AUTOFOCUS_DATA_DIR")) / "lpz_2012-2014" / "raw" / "labels.csv",
    parse_dates=["Date"],
    usecols=["Date", "Species", "filename_build"],
)

In [10]:
def get_path(filename_build):
    if filename_build.startswith("FA"):
        filename_build = Path(filename_build)
        return (
            Path(os.getenv("AUTOFOCUS_DATA_DIR"))
            / "lpz_2012-2014"
            / "raw"
            / Path(*filename_build.parts[:2])
            / "-".join(filename_build.parts[2].split("-")[:2])
            / Path(*filename_build.parts[3:])
        )
    else:
        return (
            Path(os.getenv("AUTOFOCUS_DATA_DIR"))
            / "lpz_2012-2014"
            / "raw"
            / filename_build
        )

In [11]:
df1.loc[:, "path"] = df1.loc[:, "filename_build"].progress_apply(get_path)

100%|██████████| 74560/74560 [00:02<00:00, 31247.29it/s]


In [12]:
df1 = df1.loc[df1.loc[:, "path"].apply(lambda x: x.exists()), :]

In [14]:
from PIL import Image


def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()


exif = get_exif(df1.loc[0, "path"])
print(exif)

None


In [24]:
print(Image.open(df1.loc[0, "path"])._getexif())

None


In [25]:
df1.loc[0, "path"]

PosixPath('/autofocus/data/lpz_2012-2014/raw/WI12/DPT/D10-VHC1-WI12/D10-VHC1-WI12 (23).JPG')

In [19]:
df1.loc[:, "path"].progress_apply(get_exif).notna().mean()

100%|██████████| 69346/69346 [00:10<00:00, 6524.29it/s]


0.0

In [26]:
pip install piexif


ERROR:root:Cannot parse: 1:4: pip install piexif
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/opt/conda/lib/python3.6/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "/opt/conda/lib/python3.6/site-packages/black.py", line 725, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "/opt/conda/lib/python3.6/site-packages/black.py", line 836, in lib2to3_parse
    raise exc from None
black.InvalidInput: Cannot parse: 1:4: pip install piexif


Note: you may need to restart the kernel to use updated packages.


In [33]:
import piexif

exif_dict = piexif.load(str(df1.loc[0, "path"]))
exif_dict

{'0th': {}, 'Exif': {}, 'GPS': {}, 'Interop': {}, '1st': {}, 'thumbnail': None}